# **1. Imports**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image
import glob as glob
import sys
import cv2

# **2. Environment Setup**

In [ ]:
# Path to the Environment Directory
ENV_PATH = '/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection'

# Go to the Environment Directory
os.chdir(ENV_PATH)
print(os.getcwd())

In [ ]:
# Clone TensorFlow Model Garden
if not os.path.exists('models'):
    !git clone https://github.com/tensorflow/models.git
else:
    print('Models already cloned')

# Clone Darkflow
if not os.path.exists('darkflow'):
    !git clone https://github.com/thtrieu/darkflow.git
else:
    print('Darkflow already cloned')

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!cd '/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection/models/research' && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python3 -m pip install . 

In [ ]:
# Testing if all the dependencies are installed
%cd '/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection/models/research/object_detection/builders'
!python3 model_builder_tf2_test.py

In [ ]:
%cd '/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection/models/research/object_detection'
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# **3. Loading the Model**

## 3.1 Selecting the model

In [ ]:
model_dict = {
    1: 'faster_rcnn_inception_resnet_v2_atrous',
    2: 'faster_rcnn_resnet_101',
    3: 'faster_rcnn_resnet50',
    4: 'faster_rcnn_inception_v2',
    5: 'rfcn_resnet101',
    6: 'ssd_inception_v2',
    7: 'ssd_mobilenet_v1'
}

# Chosing the model to use
model_num = int(input("Enter a model number (1-7): "))

# Validating the model number
if model_num in model_dict:
    MODEL_NAME = model_dict[model_num]
    print("Selected model:", MODEL_NAME)
else:
    print("Invalid model number.")


## 3.2 Paths

In [ ]:
MODEL_PATH = os.path.join(ENV_PATH, 'models', MODEL_NAME)
CKPT_PATH = os.path.join(MODEL_PATH, 'inference_graph/frozen_inference_graph.pb')
LABEL_PATH = os.path.join(ENV_PATH, 'scripts', 'gtsdb3_label_map.pbtxt')
NUM_CLASSES = 3

## 3.3 Load a frozen Tensorflow model

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v1.gfile.GFile(CKPT_PATH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## 3.4 Loading Label Map

In [ ]:
# Loading the label map
label_map = label_map_util.load_labelmap(LABEL_PATH)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## 3.5 Helper code

In [ ]:
# This function gets a input as an image and returns it as a NumPy array.
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
# This function preprocesses the frames.
def preprocess_frame(frame):

    # Changing the color space from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Expanding the dimensions of the frame
    frame_expanded = np.expand_dims(frame_rgb, axis=0)

    return frame_expanded

In [ ]:
# This function overlays the bounding boxes, class labels and scores on the frame.
def overlay(frame, boxes, classes, scores, box_color, box_thickness):
    # Loop over all predictions
    for i in range(boxes.shape[0]):
        # Extract the bounding box coordinates
        ymin, xmin, ymax, xmax = tuple(boxes[i].tolist())

        # Extract the class label and score
        class_label = category_index[int(classes[i])]
        score = scores[i]

        # Only draw the bounding box if the score is above 0.5
        if np.squeeze(scores)[i] > 0.5:
            # Draw the bounding box on the frame
            cv2.rectangle(frame, (int(xmin*frame.shape[1]), int(ymin*frame.shape[0])),
                        (int(xmax*frame.shape[1]), int(ymax*frame.shape[0])),
                        box_color, box_thickness)

            # Draw the class label and score on the frame
            label = "{}: {:.2f}".format(class_label['name'], score)
            label_color = (255, 255, 255) # White color
            label_bg_color = (0, 255, 0) # Green color
            label_thickness = 1
            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_thickness)[0]
            label_left = int(xmin*frame.shape[1])
            label_top = int(ymin*frame.shape[0]) - label_size[1]
            label_right = label_left + label_size[0]
            label_bottom = label_top + label_size[1]
            cv2.rectangle(frame, (label_left, label_top), (label_right, label_bottom), label_bg_color, -1)
            cv2.putText(frame, label, (label_left, label_bottom), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_color, label_thickness)

    return frame

### **EXTRA**

In [ ]:
# This code outputs all the tensors in the graph.
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        for op in detection_graph.get_operations():
            print(op.name)

# **4. Inference on Images**

## 4.1 Loading Images

In [ ]:
# Path to folder with images
TEST_IMAGES_DIR_PATH = os.path.join(ENV_PATH, 'test_images')
TEST_IMAGES_PATHS = glob.glob(os.path.join(TEST_IMAGES_DIR_PATH, '*.jpg'))

# Size of the output images
IMAGE_SIZE = (20, 20)

## 4.2 Detections

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGES_PATHS):
            image = Image.open(image_path)
            print("Processing image {}...".format(image_path))
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)

            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            print(boxes,scores, classes, num_detections)
            if np.squeeze(scores)[0] > 0.5:  # adjust the threshold as needed
                # print the number and class of objects detected
                num_objects = len(np.where(np.squeeze(scores) > 0.5)[0])
                object_classes = [category_index[c]['name'] for c in np.squeeze(classes)[:num_objects]]
                print("Detected {} objects: {}".format(num_objects, ', '.join(object_classes)))
                for i in range(num_objects):
                    # create a new figure for each object detected
                    plt.figure(idx*num_objects + i, figsize=IMAGE_SIZE)
                    plt.axis('off')
                    # crop and plot the image for the current object detected
                    ymin, xmin, ymax, xmax = tuple(boxes[0][i].tolist())
                    im_width, im_height = image.size
                    (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
                    cropped_image = image.crop((left, top, right, bottom))
                    plt.imshow(cropped_image)
                    plt.show()
                    
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=3)
            # plot the original image
            plt.figure(idx*num_objects + num_objects, figsize=IMAGE_SIZE)
            plt.axis('off')
            plt.imshow(image_np)
            plt.show()



# **5. Inference on Video**

## 5.1 Defining Input and Output Tensors

In [ ]:
# Input video tensor
frame_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
# Each box represents a part of the image where a particular object was detected.
boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
# Each score represent how level of confidence for each of the objects.
# Score is shown on the result image, together with the class label.
scores = detection_graph.get_tensor_by_name('detection_scores:0')
classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

## 5.2 Converting the file from 30 fps to 5 fps

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

PATH_TO_VIDEO = '/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection/test_video/IMG_00245fps.mp4'

# Open the video file
cap = cv2.VideoCapture(PATH_TO_VIDEO)

# Get the input video frame rate and dimensions
frame_rate = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object to write the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection/test_video/VideoTest15fps.mp4', fourcc, 5, (frame_width, frame_height))

# Initialize a frame counter and a frame skip factor
frame_counter = 0
frame_skip = int(frame_rate / 5)  # Skip frames to achieve 5 fps

# Loop over the input video frames
while cap.isOpened():
    # Read a frame from the input video
    ret, frame = cap.read()

    if not ret:
        break

    # If the current frame is a multiple of the frame skip factor, write it to the output video
    if frame_counter % frame_skip == 0:
        out.write(frame)

    # Increment the frame counter
    frame_counter += 1

# Release the input and output video objects
cap.release()
out.release()


## 5.3 Detection on video

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Initialize a frame counter
frames_counter = 0

# Paths
PATH_TO_VIDEO = '/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection/test_video/IMG_00245fps.mp4'
PATH_TO_OUTPUT_VIDEO = '/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection/results_video/IMG_0024'+str(model_num)+'new'+'.mp4'

# Open the video file
cap = cv2.VideoCapture(PATH_TO_VIDEO)

# Get the frame dimensions
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_fps = cap.get(cv2.CAP_PROP_FPS)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create a VideoWriter object to write the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(PATH_TO_OUTPUT_VIDEO, fourcc, frame_fps, (frame_width, frame_height))

with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        # Loop over all frames in the video
        while cap.isOpened():
            # Read the next frame
            ret, frame = cap.read()
            
            # If there are no more frames, break out of the loop
            if not ret:
                break

            # Print the current frame number
            frames_counter += 1
            print(f"Processing frame number: {frames_counter}/{num_frames}")

            # Preprocess the frame
            preprocessed_frame = preprocess_frame(frame)
            
            # Perform inference on the frame
            batch_boxes, batch_scores, batch_classes, batch_num_detections = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={frame_tensor: preprocessed_frame})
            
            # Overlay the predictions onto the frame
            overlayed_frame = overlay(frame, batch_boxes[0], batch_classes[0], batch_scores[0], (0,255,0), 5)

            # Overlay the number of frames processed
            cv2.rectangle(frame, (440, 40), (640, 100), (0, 0, 0), -1)
            cv2.putText(frame, str(frames_counter), (520, 90), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

            # Display the detection on each frame
            for i in range(batch_boxes[0].shape[0]):
                if np.squeeze(batch_scores[0])[i] > 0.5:
                    class_label = category_index[int(batch_classes[0][i])]
                    score = batch_scores[0][i]
                    label = "{}: {:.2f}".format(class_label['name'], score)
                    print(label)

            # Write out the frame to the output video file
            out.write(overlayed_frame)

            # Deleting all the variables
            del batch_boxes, batch_scores, batch_classes, batch_num_detections

        # Clean up
        cap.release()
        out.release()
        cv2.destroyAllWindows()

# **TESTING AREA!**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

PATH_TO_VIDEO = '/Users/henriqueoliveira18/Documents/Developer/traffic-sign-detection/test_video/VideoTest15fps.mp4'

import cv2
import numpy as np

# Load a video file
cap = cv2.VideoCapture(PATH_TO_VIDEO)

# Read the first frame
ret, frame = cap.read()

# Printing the currente frame
plt.figure(1)
plt.axis('off')
plt.imshow(frame)
plt.show()

# Preprocess the frame
frame_processed = preprocess_frame(frame)

# Printing the currente frame
plt.figure(1)
plt.axis('off')
plt.imshow(frame_processed)
plt.show()

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()